In [ ]:
# for general dataframe editing
import pandas as pd
import numpy as np

# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# for pca and pcr
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from numpy.linalg import eig



### Data Setup and Cleaning

In [ ]:
# import data
data = pd.read_csv('train.csv')
data_elements = pd.read_csv('unique_m.csv')

# look at dataset dimensions and first few observations
print(data.shape)
print(data_elements.shape)
data.head()

### Exploratory Data Analysis

In [ ]:
# list of all the predictor variables
predictors = ['number_of_elements', 'mean_atomic_mass', 'wtd_mean_atomic_mass',
       'gmean_atomic_mass', 'wtd_gmean_atomic_mass', 'entropy_atomic_mass',
       'wtd_entropy_atomic_mass', 'range_atomic_mass', 'wtd_range_atomic_mass',
       'std_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie', 'wtd_mean_fie',
       'gmean_fie', 'wtd_gmean_fie', 'entropy_fie', 'wtd_entropy_fie',
       'range_fie', 'wtd_range_fie', 'std_fie', 'wtd_std_fie',
       'mean_atomic_radius', 'wtd_mean_atomic_radius', 'gmean_atomic_radius',
       'wtd_gmean_atomic_radius', 'entropy_atomic_radius',
       'wtd_entropy_atomic_radius', 'range_atomic_radius',
       'wtd_range_atomic_radius', 'std_atomic_radius', 'wtd_std_atomic_radius',
       'mean_Density', 'wtd_mean_Density', 'gmean_Density',
       'wtd_gmean_Density', 'entropy_Density', 'wtd_entropy_Density',
       'range_Density', 'wtd_range_Density', 'std_Density', 'wtd_std_Density',
       'mean_ElectronAffinity', 'wtd_mean_ElectronAffinity',
       'gmean_ElectronAffinity', 'wtd_gmean_ElectronAffinity',
       'entropy_ElectronAffinity', 'wtd_entropy_ElectronAffinity',
       'range_ElectronAffinity', 'wtd_range_ElectronAffinity',
       'std_ElectronAffinity', 'wtd_std_ElectronAffinity', 'mean_FusionHeat',
       'wtd_mean_FusionHeat', 'gmean_FusionHeat', 'wtd_gmean_FusionHeat',
       'entropy_FusionHeat', 'wtd_entropy_FusionHeat', 'range_FusionHeat',
       'wtd_range_FusionHeat', 'std_FusionHeat', 'wtd_std_FusionHeat',
       'mean_ThermalConductivity', 'wtd_mean_ThermalConductivity',
       'gmean_ThermalConductivity', 'wtd_gmean_ThermalConductivity',
       'entropy_ThermalConductivity', 'wtd_entropy_ThermalConductivity',
       'range_ThermalConductivity', 'wtd_range_ThermalConductivity',
       'std_ThermalConductivity', 'wtd_std_ThermalConductivity',
       'mean_Valence', 'wtd_mean_Valence', 'gmean_Valence',
       'wtd_gmean_Valence', 'entropy_Valence', 'wtd_entropy_Valence',
       'range_Valence', 'wtd_range_Valence', 'std_Valence', 'wtd_std_Valence']


# making scatter plots for each predictor and the response 
plt.figure(figsize=(20,160))
for i,j in enumerate(predictors):
    plt.subplot(27,3,i+1)
    plt.scatter(data[j],data["critical_temp"], color = 'g', alpha = 0.5)
    plt.title(predictors[i])
plt.show()

In [ ]:
# looking at distribution of response variable

# setting plot style similar to R's ggplot
plt.style.use('ggplot')

# setting size of the plot
plt.figure(figsize = (8,4))

# plotting a histogram
plt.hist(data['critical_temp'].values, bins = 20,
        color = 'lightgreen',
        edgecolor = 'k')
plt.xlabel('Critical Temperature (K)')
plt.ylabel('Frequency')
plt.title('Distribution of Critical Temperatures (K)')
plt.xticks(np.arange(0, 200, step = 10));

In [ ]:
# correlation matrix
corr_mat = data.corr()
corr_mat

In [ ]:
# correlation matrix heatmap
plt.figure(figsize=(8,4))
sns.heatmap(corr_mat, cmap = 'viridis');

In [ ]:
# VIF to detect multicollinearity
X = data.drop(['critical_temp'], axis = 1)
vif = pd.DataFrame()
vif["VIF"] = np.linalg.inv(X.corr()).diagonal()
vif["features"] = X.columns
vif

### Principal Component Regression



In [ ]:
# design matrix
X = data[predictors].values

# response vector
y = data['critical_temp'].values


In [ ]:
## PCA by hand to look at eigenstructure and explained ##

# standardizing design matrix
X_std = StandardScaler().fit_transform(X)

# creating PCA object
pca = PCA()

# doing PCA to find principal components
Z = pca.fit_transform(X_std)

# creating principal components dataframe to look at
Z_df = pd.DataFrame(data = Z, columns = list(range(1,82)))
Z_df

In [ ]:
## calculating cumulative explained variance ##

# getting eigenvalues and eigenvectors of the correlation matrix of the standardized design matrix
E, V = eig(X_std.T @ X_std)
print(E[0]/np.sum(E)) # explained variance for first principal component
cumvar = np.cumsum(pca.explained_variance_ratio_) # cumulative explained variance for all principal components
cumvar

In [ ]:
# making a plot for cumulative explained variance by the number of principal components
print(cumvar[29]) # looking at cumulative explained variance for first 30 principal components
plt.style.use('default') 
plt.figure(figsize = (25,10))
plt.stem(cumvar, linefmt = 'lightgrey', markerfmt = 'navy', use_line_collection = True)
plt.xticks(range(0,81), labels = list(range(1,82)))
plt.xlabel('Number of Components', fontsize = 16)
plt.ylabel('Cumulative Explained Variance', fontsize = 16);
plt.title('Cumulative Explained Variance by Number of Components', fontsize = 20);

In [ ]:
# making scree plot - shows eigenvalues by principal components
plt.style.use('default') 
plt.figure(figsize = (25,10))
plt.plot(E)
plt.xticks(range(0,81), labels = list(range(1,82)))
plt.xlabel('Principal Component', fontsize = 16)
plt.ylabel('Coresponding Eigenvalue', fontsize = 16);
plt.title('Scree Plot', fontsize = 24);

In [ ]:
# a function to easily do PCR - source of code found in references section of paper

def pcr(predictors, response, pc):
    
    ''' Step 1: PCA on input data'''
    # Define the PCA object
    pca = PCA()
    
    # Preprocess (2) Standardize features by removing the mean and scaling to unit variance
    Xstd = StandardScaler().fit_transform(predictors)
    # Run PCA producing the reduced variable Xred and select the first pc components
    Z = pca.fit_transform(Xstd)[:,:pc]
    ''' Step 2: regression on selected principal components'''
    # Create linear regression object
    regr = LinearRegression()
    # Fit
    Z_reg = regr.fit(Z, response)
    # predicted values
    y_hat = regr.predict(Z)
    # Cross-validation
    y_cv = cross_val_predict(Z_reg, Z, response, cv=10)
    # Calculate scores for OG model and cross-validation models
    R2 = r2_score(response, y_hat)
    R2_cv = r2_score(response, y_cv)
    # Calculate mean square error for OG model and cross validation models
    mse = mean_squared_error(response, y_hat)
    mse_cv = mean_squared_error(response, y_cv)
    
    ''' Step 3: Defining coefficients of Models'''
    # coefficients of regressors for principal components regression
    E, V = eig(Xstd.T @ Xstd)
    Beta_Z = Z_reg.coef_
    Beta_X = V[:,:pc] @ Beta_Z
    
    print('R2: %5.3f'  % R2)
    print('R2 CV: %5.3f'  % R2_cv)
    print('MSE: %5.3f' % mse)
    print('MSE CV: %5.3f' % mse_cv)
    print('Intercept:', Z_reg.intercept_)
    print('Coefficients:', Z_reg.coef_)
    return(Z_reg, Beta_X, y_hat, y_cv, R2, R2_cv, mse, mse_cv)

In [ ]:
# the following several lines of code show PCR results for different subsets of principal components

Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc = 20)

In [ ]:
Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc = 22)

In [ ]:
Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc = 24)

In [ ]:
Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc = 26)

In [ ]:
Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc = 28)

In [ ]:
Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc = 30)

In [ ]:
Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc = 32)

In [ ]:
# just to see what the coefficients look like after transformed back to standardized design matrix units
print(Z_model.intercept_)
print(beta_X)

In [ ]:
# for final model and for plotting
Z_model, beta_X, predicted, predicted_cv, r2, r2_cv, mse, mse_cv = pcr(X, y, pc=30)


# fitting a regression line for plotting
z = np.polyfit(y, predicted, 1)

# plotting the predicted values by the observed values to show how well the model predicted critical temperature and also
# to plot the regression line
with plt.style.context(('ggplot')):
    fig, ax = plt.subplots(figsize = (9, 5))
    ax.scatter(y, predicted, c = 'lightcyan', edgecolors = 'k')
    ax.plot(y, y, c = 'yellow', linewidth = 2, label = 'Observed Tc') # can't make a dashed line for reasons I do not understand
    ax.plot(y, z[1] + z[0]*y, c ='blue', linewidth = 2, label = 'Predicted Tc')
    plt.title('Cross-validation R2: ' + str(round(r2_cv,3)))
    plt.xlabel('Observed')
    plt.ylabel('Predicted')
    ax.legend(loc = 'upper left', facecolor = 'white');
    plt.show()

In [ ]:
# a function I either made myself or found (can't remember or find source) to find calculate five-number summary of a vector
def summary_stats(data):

    # calculate quartiles
    quartiles = np.percentile(data, [25, 50, 75])
    # calculate min/max
    data_min, data_max = data.min(), data.max()
    # print 5-number summary
    print('Min: %.3f' % data_min)
    print('Q1: %.3f' % quartiles[0])
    print('Median: %.3f' % quartiles[1])
    print('Q3: %.3f' % quartiles[2])
    print('Max: %.3f' % data_max)


# comparing five-number summaries between observed and predicted values
summary_stats(predicted)
summary_stats(y)

In [ ]:
## plots for residual analysis ## 
  
# setting plot style 
plt.style.use('ggplot') 
  
# plotting residual errors 
plt.figure(figsize = (8,5))
plt.scatter(predicted, y - predicted, 
            color = "green", s = 10)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')

# plot title 
plt.title("Residual Plot", fontsize = 16); 

In [ ]:
# qq-plot of residuals
plt.figure(figsize = (8,5))
stats.probplot(y - predicted, dist="norm", plot=plt)
plt.title("Normal Q-Q Plot");

In [ ]:
# this cell contains code that was used to compare particular elements one at a time
# notice that I did not drop the 'Oxygen' ('O') element from the dataset 'data' - want to use it for regression analysis and
# to demonstrate using an indicator variable with PCR

data_elements = pd.read_csv('unique_m.csv')
data = pd.read_csv('train.csv')

data_elements = data_elements.drop(['critical_temp'], axis = 1)
data = pd.concat([data, data_elements], axis = 1)
data = data.drop(['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'F', 'Ne','Na', 
                  'Mg', 'Al', 'Si', 'P', 'S', 'Cl','Ar', 'K','Ca', 'Sc', 'Ti', 
                  'V', 'Cr', 'Mn', 'Pb', 'Co','Ni', 'Cu', 'Zn', 'Ga', 'Ge', 
                  'As', 'Se', 'Br', 'Kr', 'Rb','Sr', 'Y', 'Zr', 'Nb', 'Mo', 
                  'Tc', 'Ru', 'Rh', 'Pd', 'Ag','Cd', 'In', 'Sn', 'Sb', 'Te', 
                  'I', 'Xe', 'Cs', 'Ba', 'La','Ce', 'Pr', 'Nd', 'Pm', 'Sm', 
                  'Eu', 'Gd', 'Tb', 'Dy', 'Ho','Er', 'Tm', 'Yb', 'Lu', 'Hf', 
                  'Ta', 'W', 'Re', 'Os', 'Ir','Pt', 'Au', 'Hg', 'Tl', 'Fe', 
                  'Bi', 'Po', 'At', 'Rn', 'material'], axis = 1)

data_elements = data_elements[['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al',
       'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn',
       'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
       'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
       'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm',
       'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
       'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At',
       'Rn']]

data

In [ ]:
# another five-number summary function (I definitely made this one myself) specifically to use on the elements dataset
# I just wanted to get a sense of how many superconductors had which elements and how much of that element they had

def summary_stats(df):
    '''seven number summary of a numerical vector'''
    global summary
    names = ['Size','Min','25th Percentile','Median','75th Percentile','Max','Mean','Standard Deviation']
    summary = pd.DataFrame(columns = names)
    elements = []
    for element in df.columns.values:
        if df[element].values.max() != 0:
            elements.append(element)
            x = df[df[element].values > 0]
            summaries = [len(x[element].values), round(x[element].values.min(),4), 
                         round(np.percentile(x[element].values, 25),4), 
                         round(np.percentile(x[element].values, 50),4), 
                         round(np.percentile(x[element].values, 75),4), 
                         round(x[element].values.max(),4), 
                         round(x[element].values.mean(),4), 
                         round(x[element].values.std(),4)]
            summaries = pd.DataFrame([summaries], columns = names)
            summary = summary.append(summaries)
    summary = summary.set_index([elements])
    return summary

# it shows the number of super conductors that contain a particular element 'Size' along with the five-number summary
# for instance, 299 superconductors contained Hydrogen and the superconductor that had the 
# most hydrogen had 14 (not sure about units)

summary_stats(data_elements)
summary.head()

In [ ]:
# looking at all elements what were found in more than 2,000 superconductors
# this is how I decided which elements to play with, eventually choosing Oxygen because of its clear PCA score plot
summary[summary.Size > 2000]

In [ ]:
# restructuring data to do PCA and eventually introduce indicator variable
dataO = data[data['O'] > 0].reset_index()
yO = dataO['critical_temp']
dataO = dataO.drop(['index','critical_temp', 'O'], axis = 1)

dataNO = data[data['O'] == 0].reset_index()
yNO = dataNO['critical_temp']
dataNO = dataNO.drop(['index','critical_temp', 'O'], axis = 1)

y = pd.DataFrame(yNO)
y = y.append(pd.DataFrame(data = yO), ignore_index = True)

X_model = dataNO.append(dataO, ignore_index = True)

# making oxygen into an indicator variable through factorization
Oxygen0 = data['O'][data['O'] == 0].reset_index().drop(['index'], axis = 1)
Oxygen1 = pd.DataFrame({'O': np.ones(11964)})

Oxygen_edit = Oxygen0.append(Oxygen1, ignore_index = True)
Oxygen = Oxygen_edit.apply(lambda x: x.factorize()[0])

len(Oxygen[Oxygen['O'] == 1]) # correct size

In [ ]:
## PCR by hand to carefully account for indicator variable ##

# standardizing data
X_std = StandardScaler().fit_transform(X_model)

# creating PCA object
pca = PCA()

# principal components
Z = pca.fit_transform(X_std)[:,:30]

# creating principal components dataframe to look at
Z_pca = pd.DataFrame(data = Z, columns = list(range(1,31)))

# introducing indicator variable
Z_df = pd.concat([Oxygen, Z_pca], axis = 1)

# Create linear regression object
regr = LinearRegression()

# Fit
Z_reg = regr.fit(Z_df, y)

# predicted values
y_hat = regr.predict(Z_df)

# Cross-validation
y_cv = cross_val_predict(Z_reg, Z_df, y, cv=10)

# Calculate scores for OG model and cross-validation models
R2 = r2_score(y, y_hat)
R2_cv = r2_score(y, y_cv)

# Calculate mean square error for OG model and cross validation models
mse = mean_squared_error(y, y_hat)
mse_cv = mean_squared_error(y, y_cv)
    
print('R2: %5.3f'  % R2)
print('R2 CV: %5.3f'  % R2_cv)
print('MSE: %5.3f' % mse)
print('MSE CV: %5.3f' % mse_cv)
print('Intercept:', Z_reg.intercept_)
print('Coefficients:', Z_reg.coef_)

In [ ]:
# making observed values into a usable vector
yvec = y['critical_temp']

# making sure I subset the observed and predicted values correctly
print(len(yvec[:9299])) # number of observations for super conductors without oxygen = 9299
print(len(yvec[9299::])) # number of observations for super conductors with oxygen = 11964
print(len(yvec[:9299]) + len(yvec[9299::])) # total observations

In [ ]:

# fitting regression lines for plotting
z0 = np.polyfit(yvec[:9299], y_hat[:9299], 1)
z1 = np.polyfit(yvec[9299::], y_hat[9299::], 1)

# plotting the predicted values by the observed values to show how well the model predicted critical temperature and also
# to plot the regression lines
with plt.style.context(('ggplot')):
    fig, ax = plt.subplots(figsize = (10, 6))
    ax.scatter(yvec[9299::], y_hat[9299::], c = 'lightcyan', edgecolors = 'k', label = 'With oxygen')
    ax.scatter(yvec[:9299], y_hat[:9299], c = 'mistyrose', edgecolors = 'k', label= 'Without oxygen')
    ax.plot(yvec, yvec, c ='yellow', linewidth = 2, label = 'Observed Tc') # still can't coerce a dashed line
    ax.plot(yvec[:9299], z0[1] + z0[0]*yvec[:9299], c ='red', linewidth = 2, label = 'Without oxygen, predicted Tc')
    ax.plot(yvec[9299::], z1[1] + z1[0]*yvec[9299::], c ='blue', linewidth = 2, label = 'With oxygen, predicted Tc')
    plt.title('Cross-validation R2: ' + str(round(R2_cv,3)))
    plt.xlabel('Observed')
    plt.ylabel('Predicted')
    ax.legend(loc = 'upper left', facecolor = 'white');
    plt.show()

In [ ]:
## plots for residual analysis ##
  
# setting plot style 
plt.style.use('ggplot') 
  
# plotting residual errors 
plt.figure(figsize = (8,5))
plt.scatter(predicted, yvec - predicted, 
            color = "green", s = 10)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')

# plot title 
plt.title("Residual Plot", fontsize = 16); 

In [ ]:
# qq-plot of residuals
plt.figure(figsize = (8,5))
stats.probplot(yvec - predicted, dist="norm", plot=plt)
plt.title("Normal Q-Q Plot");